![encodelogo](images/encodelogo.gif)

# Exploration of ENCODE cell types

This notebook will explore ENCODE [CTCF](https://www.ncbi.nlm.nih.gov/gene/10664) ChIP-seq data across different cell types. We will first narrow down the ENCODE data to explore by querying the [ENCODE Portal](https://www.encodeproject.org) using its [REST API](https://www.encodeproject.org/help/rest-api/). Then, we will process the data into a more manageable format before visualizing cross-cell-type Spearman correlations.

This notebook has been tested using Python 3.7. If you are using the Anaconda distribution, you should only need to install *deepTools* via `pip install deeptools`. 

In [1]:
%matplotlib inline

import deeptools.getScorePerBigWigBin as score_bw
from deeptools.correlation import Correlation
import numpy as np
import matplotlib.pyplot as plt
import os
import pprint
import pandas as pd
import requests
from urllib.parse import urlparse

## Obtain ENCODE metadata
Here is a [link](https://www.encodeproject.org/search/?type=Experiment&status=released&assay_title=ChIP-seq&biosample_ontology.classification=cell+line&replicates.library.biosample.donor.organism.scientific_name=Homo+sapiens&target.investigated_as=transcription+factor&replicates.library.biosample.treatments%21=%2A&award.project=ENCODE&award.rfa=ENCODE3&target.label=CTCF) to a small search set of ENCODE CTCF ChIP-seq experiments on untreated samples in various cell lines. Using the REST API and the [ENCODE experiment search](https://www.encodeproject.org/search/?type=Experiment) we can obtain the metadata for these experiments programatically, including more information about the cell lines that were used. The *frame=embedded* query parameter allows us to obtain a full expansion of the embedded *biosample_ontology* objects containing biosample metadata of interest. Searches from the website appear in the URL of the browser, which can be copy-pasted into the script below (don't forget to add *frame=embedded* and *format=json*).

In [13]:
url = (
    'https://www.encodeproject.org/search/?type=Experiment'
    '&status=released'
    '&assay_title=DNase-seq'
    '&biosample_ontology.classification=tissue'
    '&replicates.library.biosample.donor.organism.scientific_name=Homo+sapiens'
    '&replicates.library.biosample.treatments!=*'
    '&award.project=ENCODE'
    '&award.rfa=ENCODE3'
    '&frame=embedded'
    '&limit=all'
    '&internal_tags=ENTEx'
    '&format=json'
)
r = requests.get(url)
experiments = r.json()['@graph']
# Obtain a subset of the biosample metadata.
biosample_ontologies = []
for e in experiments:
    # Get the name of the cell line, as well as more
    # general categorical information.
    biosample_ontology_condensed = {
        k: v
        for k, v in e.get('biosample_ontology', {}).items()
        if k == 'term_name' or k.endswith('_slims')
    }
    biosample_ontology_condensed.update(
        {'dataset': e.get('@id')}
    )
    biosample_ontologies.append(biosample_ontology_condensed)
biosample_metadata = pd.DataFrame(biosample_ontologies)
biosample_metadata

,cell_slims,dataset,developmental_slims,organ_slims,system_slims,term_name
0,[],/experiments/ENCSR322UXF/,[endoderm],"[intestine, lymphoid tissue]","[digestive system, immune system]",Peyer's patch
1,[],/experiments/ENCSR979ZJS/,[endoderm],"[intestine, colon, large intestine]",[digestive system],transverse colon
2,[],/experiments/ENCSR080ISA/,[mesoderm],"[limb, arterial blood vessel, blood vessel, va...",[circulatory system],tibial artery
3,[],/experiments/ENCSR958QXU/,"[endoderm, ectoderm, mesoderm]",[prostate gland],[reproductive system],prostate gland
4,[],/experiments/ENCSR852TRT/,[mesoderm],"[arterial blood vessel, blood vessel, vasculat...",[circulatory system],ascending aorta
5,[],/experiments/ENCSR864IGD/,[ectoderm],"[limb, skin of body]",[integumental system],lower leg skin
6,[],/experiments/ENCSR158VAT/,[endoderm],[thyroid gland],[endocrine system],thyroid gland
7,[],/experiments/ENCSR848XIY/,"[mesoderm, ectoderm]",[adrenal gland],[endocrine system],adrenal gland
8,[],/experiments/ENCSR228IKB/,[],[pancreas],[endocrine system],body of pancreas
9,[],/experiments/ENCSR163PKT/,[endoderm],[stomach],[digestive system],stomach


The information we have gathered here will hopefully give us some biological intuition as we explore the data. Next, we will use this list of experiments to gather a set of bigWig files that we will use for our analysis, again using the REST API. Here, we are interested in released signal p-value files from pooled replicates processed on GRCh38. We need to pass the query parameter `limit=all` to return the full list of results, otherwise it will be truncated to the first 25 results.

In [14]:
# Parse the experiments into query parameters.
datasets = [
    '&dataset={}'.format(e.get('@id'))
    for e in experiments
]
url = (
    'https://www.encodeproject.org/search/?type=File'
    '&status=released'
    '&output_type=read-depth normalized signal'
    '&assembly=GRCh38'
    '&field=dataset'
    '&field=biological_replicates'
    '&field=cloud_metadata.url'
    '&frame=object'
    '&format=json'
    '&limit=all'
    '&{}'.format(''.join(datasets))
)
r = requests.get(url)
# Filter the files to only those belonging to multiple replicates.
#files = [
#    f
#    for f in r.json()['@graph']
#    if len(f['biological_replicates']) > 1
#]
file_metadata = pd.DataFrame(r.json()['@graph'])
# Flatten JSON.
file_metadata['cloud_metadata'] = file_metadata.cloud_metadata.apply(lambda x: x.get('url'))
# Rename column.
file_metadata = file_metadata.rename(columns={'@id': 'file'})
# Join with biosample_metadata.
merged_df = file_metadata.merge(biosample_metadata, how='outer', on='dataset')
merged_df = merged_df[[
    'file',
    'dataset',
    'cell_slims',
    'developmental_slims',
    'organ_slims',
    'system_slims',
    'term_name',
    'biological_replicates',
    'cloud_metadata'
]]
merged_df

,file,dataset,cell_slims,developmental_slims,organ_slims,system_slims,term_name,biological_replicates,cloud_metadata
0,/files/ENCFF102SIK/,/experiments/ENCSR228IKB/,[],[],[pancreas],[endocrine system],body of pancreas,[1],https://encode-public.s3.amazonaws.com/2017/12...
1,/files/ENCFF233ORX/,/experiments/ENCSR852TRT/,[],[mesoderm],"[arterial blood vessel, blood vessel, vasculat...",[circulatory system],ascending aorta,[1],https://encode-public.s3.amazonaws.com/2017/09...
2,/files/ENCFF376RIO/,/experiments/ENCSR783OCW/,[],[endoderm],"[epithelium, esophagus]",[digestive system],esophagus squamous epithelium,[1],https://encode-public.s3.amazonaws.com/2017/09...
3,/files/ENCFF414NAA/,/experiments/ENCSR164WOF/,[],[endoderm],[lung],[respiratory system],upper lobe of left lung,[1],https://encode-public.s3.amazonaws.com/2017/09...
4,/files/ENCFF098CAJ/,/experiments/ENCSR790FIS/,[],[endoderm],"[intestine, colon, large intestine]",[digestive system],transverse colon,[1],https://encode-public.s3.amazonaws.com/2017/08...
5,/files/ENCFF799HNC/,/experiments/ENCSR452ZMO/,[],[endoderm],"[lymphoid tissue, intestine]","[immune system, digestive system]",Peyer's patch,[1],https://encode-public.s3.amazonaws.com/2017/12...
6,/files/ENCFF762FMJ/,/experiments/ENCSR902XFY/,[],[endoderm],[thyroid gland],[endocrine system],thyroid gland,[1],https://encode-public.s3.amazonaws.com/2018/02...
7,/files/ENCFF561DJM/,/experiments/ENCSR864IGD/,[],[ectoderm],"[limb, skin of body]",[integumental system],lower leg skin,[1],https://encode-public.s3.amazonaws.com/2017/09...
8,/files/ENCFF415UET/,/experiments/ENCSR848XIY/,[],"[mesoderm, ectoderm]",[adrenal gland],[endocrine system],adrenal gland,[1],https://encode-public.s3.amazonaws.com/2017/08...
9,/files/ENCFF412KRU/,/experiments/ENCSR777DXQ/,[],[mesoderm],[heart],[circulatory system],right atrium auricular region,[1],https://encode-public.s3.amazonaws.com/2017/09...


## Download files
Note that in the `cloud_metadata.url` field is a direct HTTPS link to the file object in *s3://encode-public*. Here we will use `wget`, but it is also possible to obtain the files via `aws s3 cp` or by using `boto3` in Python. As a cloud alternative to downloading the files see [how to mount *s3://encode-public* on an EC2 instance](https://github.com/ENCODE-DCC/encode-data-usage-examples/blob/master/mount_s3_bucket_and_run_jupyter_on_ec2.ipynb).

In [16]:
len(merged_df)

56

In [37]:
# Uncomment to download files to data/.
os.system('mkdir data')
for url in merged_df.cloud_metadata.values:
    os.system('cd data && wget {}'.format(url))

## Analyze the data
Now that we have the files, we can compute correlations across all of the datasets, which in this case will result in a 24 by 24 symmetric matric. Using the *deepTools* library, we will first compute average scores across 10kb regions, then compute and plot correlations on the averaged data. This workflow has been adapted from the *deepTools* *plotCorrelation* and *multiBigwigSummary* Python scripts using components of the [deepTools API](https://deeptools.readthedocs.io/en/develop/content/api.html). There is nothing significant about the 10kb regions (try changing this parameter and see what happens).

In [39]:
def make_corr_plot(
        bw_files,
        chrs_to_skip,
        bin_size=10000,
        method='pearson',
        fileset_name='result',
        blacklist=None,
        labels=None,
        output_dir=''
    ):
    """ 
    A two-step workflow approximating the sequential usage of deepTools
    command line tools multiBigWigSummary and plotCorrelation. The bigWig
    files are binned (10kb regions by default) and saved as .npz files
    before correlation matrices are computed and plots saved as png images.
    
    Parameters
    ----------
    bw_files : list
        A list of paths to bigWig files
    chrs_to_skip : list
        A list of chromosomes to skip during correlation computation
    bin_size : int (optional)
        Size of bins (in base pairs) in which to bin the bigWig files before
        correlations are computed
    method : str (optional)
        Name of the correlation function to use, must be one of 'pearson'
        or 'spearman'
    fileset_name : str (optional)
        Name describing the analysis. This will appear in the plot title
    blacklist : str (optional):
        Path to a .bed file of blacklisted regions to exclude from
        correlations
    labels : list (optional)
        List of meaningful descriptions for each file, these will be used
        to label the rows and columns of the output matrix plot
    output_dir : str (optional)
        Directory where the .npz file and output plot will be saved
    """
    assert method in ('pearson', 'spearman'), 'Invalid correlation method'
    # Autogenerate labels from filename if not provided
    if not labels:
        labels = [filename.split('/')[-1].split('.')[0] for filename in bw_files]
    # Generate a name for the unique combination
    test_name = fileset_name + '_' + method
    if blacklist:
        blacklist_title = 'Blacklisted '
        test_name += '_blacklisted'
    else:
        blacklist_title = ''
    image_name = test_name + '.png'
    # Bin the bigwig data in 10kb increments
    num_reads_per_bin = score_bw.getScorePerBin(
        bw_files,
        bin_size,
        chrsToSkip=chrs_to_skip,
        blackListFileName=blacklist
    )
    # Write to npz file
    filename = output_dir + test_name + '.npz'
    with open(filename, "wb") as f:
        np.savez_compressed(f, matrix=num_reads_per_bin, labels=labels)
    # Compute the correlations
    corr = Correlation(filename, method, labels=labels)
    plot_title = '{}{} Correlation of {}'.format(
        blacklist_title,
        method.capitalize(),
        fileset_name
    )
    # Create a png file of correlation heatmap
    image_path = output_dir + image_name
    corr.plot_correlation(image_path, plot_title=plot_title)
    return image_path

Now we need to prepare arguments to this function. We will compute correlations only on chromosome 1 (chr1). In order to do so, we need to pass a list of chromosomes to skip. We can use the [GRCh38 chrom.sizes file](https://www.encodeproject.org/files/GRCh38_EBV.chrom.sizes/), to find the names of chromosomes and scaffolds that we would like to skip. It is available on S3 [here](https://encode-public.s3.amazonaws.com/2016/01/06/89effdbe-9e3f-48c6-9781-81e565ac45a3/GRCh38_EBV.chrom.sizes.tsv). 

In [40]:
%%bash
mkdir output_images
cd data
wget https://encode-public.s3.amazonaws.com/2016/01/06/89effdbe-9e3f-48c6-9781-81e565ac45a3/GRCh38_EBV.chrom.sizes.tsv
ls


ENCFF004TQX.bigWig
ENCFF016FKV.bigWig
ENCFF019UDG.bigWig
ENCFF040ORD.bigWig
ENCFF059EPZ.bigWig
ENCFF111QNV.bigWig
ENCFF134CFC.bigWig
ENCFF146RCM.bigWig
ENCFF156PPR.bigWig
ENCFF166JWH.bigWig
ENCFF195CGX.bigWig
ENCFF252TXI.bigWig
ENCFF289MVP.bigWig
ENCFF318ASK.bigWig
ENCFF325GYQ.bigWig
ENCFF345AEO.bigWig
ENCFF411YJS.bigWig
ENCFF436WYT.bigWig
ENCFF457PEW.bigWig
ENCFF488ISB.bigWig
ENCFF494OPQ.bigWig
ENCFF509NMU.bigWig
ENCFF539QXW.bigWig
ENCFF543QNU.bigWig
ENCFF571JGH.bigWig
ENCFF574PNA.bigWig
ENCFF632AHW.bigWig
ENCFF638XFB.bigWig
ENCFF651PRW.bigWig
ENCFF677UYW.bigWig
ENCFF725OGN.bigWig
ENCFF727WHR.bigWig
ENCFF733AOC.bigWig
ENCFF769KKD.bigWig
ENCFF799SEU.bigWig
ENCFF802BAX.bigWig
ENCFF831DQC.bigWig
ENCFF850SWF.bigWig
ENCFF860ISR.bigWig
ENCFF915YAY.bigWig
ENCFF952TLY.bigWig
ENCFF959XTO.bigWig
GRCh38_EBV.chrom.sizes.tsv
GRCh38_EBV.chrom.sizes.tsv.1
GRCh38_EBV.chrom.sizes.tsv.2


mkdir: output_images: File exists
--2019-04-09 19:05:22--  https://encode-public.s3.amazonaws.com/2016/01/06/89effdbe-9e3f-48c6-9781-81e565ac45a3/GRCh38_EBV.chrom.sizes.tsv
Resolving encode-public.s3.amazonaws.com... 52.218.224.122
Connecting to encode-public.s3.amazonaws.com|52.218.224.122|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11686 (11K) [binary/octet-stream]
Saving to: ‘GRCh38_EBV.chrom.sizes.tsv.2’

     0K .......... .                                          100% 14.0M=0.001s

2019-04-09 19:05:23 (14.0 MB/s) - ‘GRCh38_EBV.chrom.sizes.tsv.2’ saved [11686/11686]



In [41]:
chrom_sizes = pd.read_csv('data/GRCh38_EBV.chrom.sizes.tsv', sep='\t', header=None)
chrom_sizes

,0,1
0,chr1,248956422
1,chr2,242193529
2,chr3,198295559
3,chr4,190214555
4,chr5,181538259
5,chr6,170805979
6,chr7,159345973
7,chr8,145138636
8,chr9,138394717
9,chr10,133797422


We can see the huge number of scaffolds that don't pertain to a particular chromosome. We will leave these out of our analysis by adding them to the list of chromosomes to skip.

In [42]:
%matplotlib inline
data_dir = 'data/'
# Create set of all chromosomes in GRCh38
chr_labels = set(chrom_sizes[0].tolist())
# Create set with desired label
all_chrs = [ ]
desired_chrs = {'chr'+str(x) for x in (range(1,22)) }
# Using set difference create a list of chromosomes to skip
chrs_to_skip = list(chr_labels - desired_chrs)
# Use the Spearman correlation for comparison, it is less susceptible to outliers than Pearson correlation
method = 'pearson'
# Generate the local bigWig file paths and cell line labels
bw_files = []
bw_labels = []
for f in merged_df.to_dict('record'):
    bw_file = data_dir + os.path.basename(f.get('cloud_metadata'))
    bw_label = f.get('term_name')
    bw_files.append(bw_file)
    bw_labels.append(bw_label)
imagename = make_corr_plot(
    bw_files,
    chrs_to_skip,
    method=method,
    fileset_name='H3k27me3_ChIP-seq',
    labels=bw_labels,
    output_dir='output_images/'
)
print(imagename)


The following chromosome names did not match between the the bigwig files
chromosome	length
chrUn_KI270580v1	      1553
chrUn_KI270305v1	      1472
chrUn_KI270529v1	      1899
chrUn_KI270310v1	      1201
chrUn_KI270579v1	     31033
chrY_KI270740v1_random	     37240
chrUn_KI270423v1	       981
chrUn_KI270748v1	     93321
chrUn_KI270395v1	      1143
chrUn_KI270378v1	      1048
chrUn_KI270425v1	      1884
chrUn_KI270396v1	      1880
chrUn_KI270581v1	      7046
chrUn_KI270530v1	      2168
         chrEBV	    171823
chrUn_KI270322v1	     21476
chrUn_KI270315v1	      2276
chrUn_KI270330v1	      1652
chrUn_KI270302v1	      2274
chrUn_KI270382v1	      4215
chrUn_KI270362v1	      3530
chr2_KI270715v1_random	    161471
chrUn_KI270419v1	      1029
chrUn_KI270363v1	      1803
chrUn_KI270392v1	       971
chrUn_KI270320v1	      4416
chrUn_KI270544v1	      1202
chrUn_KI270317v1	     37690
chrUn_KI270418v1	      2145
chrUn_KI270412v1	      1179
chrUn_KI270752v1	     27745
chrUn_KI270424v1	      2140


output_images/CTCF_ChIP-seq_pearson.png


## Plot the results
Now that we have our correlation plot, let's display it. Conveniently, the *deepTools* *plot_correlation* method plots hierarchical clustering results along one end of the matrix. 

In [43]:
fig, ax = plt.subplots(1, 1)
scale = 40
img = plt.imread(imagename)
imgplot = ax.imshow(img)
fig.set_size_inches(tuple([scale*i for i in fig.get_size_inches()]))

The most correlated cell types here are RWPE1, C4-2B, and LNCAP. What do these cell lines have in common?

In [51]:
interesting_lines = ['SU-DHL-6','Karpas-422','OCI-LY1']
merged_df[merged_df.term_name.isin(interesting_lines)]

,file,dataset,cell_slims,developmental_slims,organ_slims,system_slims,term_name,biological_replicates,cloud_metadata
0,/files/ENCFF860ISR/,/experiments/ENCSR963HAR/,"[hematopoietic cell, B cell, leukocyte, cancer...",[mesoderm],"[blood, lymph node, bodily fluid]",[immune system],Karpas-422,"[1, 2]",https://encode-public.s3.amazonaws.com/2016/10...
1,/files/ENCFF134CFC/,/experiments/ENCSR859JNA/,[cancer cell],[mesoderm],"[bone marrow, bone element]","[skeletal system, immune system]",OCI-LY1,"[1, 2]",https://encode-public.s3.amazonaws.com/2016/10...
14,/files/ENCFF156PPR/,/experiments/ENCSR511OIE/,[cancer cell],[],[],[],SU-DHL-6,"[1, 2]",https://encode-public.s3.amazonaws.com/2017/01...


Cool! These all appear to be prostate cell lines, which could explain their high correlation. Interestingly though, other prostate cell lines, like 22rv1 and RWPE2 don't appear as part of this cluster. We also see a broad and loosely defined cluster around *Loucy* (darker red than other clusters). Let's look at a few cell lines there as well:

In [52]:
interesting_lines = ['GM23248','ACC112']
merged_df[merged_df.term_name.isin(interesting_lines)]

,file,dataset,cell_slims,developmental_slims,organ_slims,system_slims,term_name,biological_replicates,cloud_metadata
3,/files/ENCFF574PNA/,/experiments/ENCSR866KFY/,[fibroblast],[ectoderm],"[connective tissue, skin of body, limb]",[integumental system],GM23248,"[1, 2]",https://encode-public.s3.amazonaws.com/2016/11...
8,/files/ENCFF959XTO/,/experiments/ENCSR475NLR/,"[epithelial cell, cancer cell]","[endoderm, ectoderm]","[epithelium, mouth]",[digestive system],ACC112,"[1, 2]",https://encode-public.s3.amazonaws.com/2016/11...


We can see that these are all cancer cells. We can see numerous other cancer cell lines in this cluster as well, such as OCI-LY7 and PC-9.

## Conclusions
Here are some exercises left to the reader:

1. How would this analysis look on a different chromosome? Does it change meaningfully? How would it look if we performed it genome-wide?

2. How would this analysis look if we used ENCODE ChIP-seq data for a different transcription factor? Would we expect similar clusters to form?  What about a histone mark like H3K4me3?

3. Did the results here match our expectations for CTCF? What biological knowledge about CTCF could help explain these results?